## Icolos Ensemble MM-GBSA workflow
In this notebook, we demonstrate an Icolos workflow for automated mm-gbsa calculations for a protein-ligand complex using GROMACS.  The required files are the `mdp` files containing the run settings for each stage, and a prepared pdb file containg the protein-ligand complex suitable for gromacs.  Ligands and cofactors are automatically parametrised with GAFF through the `acpype` wrapper for Antechamber.

Files required to execute the workflow are provided in the accompanying IcolosData repository, available at [here](https://github.com/MolecularAI/IcolosData), and we use the Icolos environment from the main package [here](https://github.com/MolecularAI/Icolos)

Please use the `icolosprod` environment specified in the main repository

In [5]:
import os
import json
import subprocess
import pandas as pd

# set up some file paths to use the provided test data
# please ammend as appropriate
icolos_path = "~/Icolos"
data_dir = "~/IcolosData"
output_dir = os.path.abspath("../output")
# set your forcefield dir
os.environ["GMXLIB"]= "<forcefield_path>"
forcefield_path = os.environ.get("GMXLIB")

config_dir = "../config/gromacs"
for path in [output_dir, config_dir]:
    os.makedirs(path, exist_ok=True)
pdb_path = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/1BVG.pdb"))
ions_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/ions.mdp"))
em_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/minim.mdp"))
nvt_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/nvt_equil.mdp"))
npt_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/npt_equil.mdp"))
prod_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/md.mdp"))




In [6]:
conf = {
	"workflow": {
		"header": {
			"workflow_id": "gromacs_ensemble_mmgbsa",
			"description": "ensemble MMGBSA demonstration - step iteration + SLURM job control",
			"environment": {
				"export": [
					{
						"key": "GMX_GPU_DD_COMMS",
						"value": "true"
					},
					{
						"key": "GMX_GPU_PME_PP_COMMS",
						"value": "true"
					},
					{
						"key": "GMX_FORCE_UPDATE_DEFAULT_GPU",
						"value": "true"
					}
				]
			},
			"global_variables": {
			}
		},
		"steps": [
			{
				"step_id": "01_pdb2gmx",
				"type": "pdb2gmx",
				"execution": {
					"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
				},
				"settings": {
					"arguments": {
						"flags": [
							"-ignh"
						],
						"parameters": {
							"-water": "tip3p",
							"-ff": "amber14sb"
						}
					},
					"additional": {
                        "charge_method": "gas"
                    }
				},
				"input": {
					"generic": [
						{
							"source": pdb_path,
							"extension": "pdb"
						}
					]
				}
			},
			{
				"step_id": "02_editconf",
				"type": "editconf",
				"execution": {
					"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
				},
				"settings": {
					"arguments": {
						"flags": [
							"-c"
						],
						"parameters": {
							"-d": "1.2",
							"-bt": "dodecahedron"
						}
					},
					"additional": {}
				}
				
			},
			{
				"step_id": "03_solvate",
				"type": "solvate",
				"execution": {
					"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
				},
				"settings": {
					"arguments": {
						"flags": [],
						"parameters": {
							"-cs": "spc216.gro"
						}
					},
					"additional": {}
				}
				
			},
			{
				"step_id": "04_grompp",
				"type": "grompp",
				"execution": {
					"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
				},
				"settings": {
					"arguments": {
						"flags": [],
						"parameters": {}
					},
					"additional": {
						"-r":False
					}
				},
				"input": {
					"generic": [
						{
							"source":ions_mdp,
							"extension": "mdp"
						}
					]
				}
			},
			{
				"step_id": "05_genion",
				"type": "genion",
				"execution": {
					"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
				},
				"settings": {
					"arguments": {
						"flags": [
							"-neutral"
						],
						"parameters": {
							"-pname": "NA",
							"-nname": "CL"
						}
					},
					"additional": {
						"pipe_input": "SOL"
					}
				},
				"input": {
					"generic": [
						{
							"source": "04_grompp",
							"extension": "tpr"
						}
					]
				}
			},
			{
				"step_id": "gromacs_iterator",
				"type": "iterator",
				"base_config": [
					{
						"step_id": "06_grompp_eminim",
						"type": "grompp",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {}
							},
							"additional": {
								"-r":False
							}
						},
						"input": {
							"generic": [
								
								{
									"source":em_mdp,
									"extension": "mdp"
								}
							]
						}
					},
					{
						"step_id": "07_eminim_mdrun",
						"type": "mdrun",
						"execution": {
							"platform": "slurm",
							"resources": {
								"partition": "gpu",
								"gres": "gpu:1",
								"modules": [
									"GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
								]
							}
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {}
							},
							"additional": {}
						},
						"input": {
							"generic": [
								{
									"source": "06_grompp_eminim",
									"extension": "tpr"
								}
							]
						}
					},
					{
						"step_id": "08_nvt_grompp",
						"type": "grompp",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {
									"-n": "index.ndx"
								}
							},
							"additional": {
								"make_ndx_command": "auto",
								"-r":True
							}
						},
						"input": {
							"generic": [
								
								{
									"source":nvt_mdp,
									"extension": "mdp"
								}
							]
						}
					},
					{
						"step_id": "09_nvt_mdrun",
						"type": "mdrun",
						"execution": {
							"platform": "slurm",
							"resources": {
								"partition": "gpu",
								"gres": "gpu:1",
								"modules": [
									"GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
								]
							}
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {}
							},
							"additional": {}
						},
						"input": {
							"generic": [
								{
									"source": "08_nvt_grompp",
									"extension": "tpr"
								}
							]
						}
					},
					{
						"step_id": "10_npt_grompp",
						"type": "grompp",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {
									"-n": "index.ndx"
								}
							},
							"additional": {
								"make_ndx_command": "auto",
								"-r":True
							}
						},
						"input": {
							"generic": [
								
								{
									"source":npt_mdp,
									"extension": "mdp"
								}
								
							]
						}
					},
					{
						"step_id": "11_npt_mdrun",
						"type": "mdrun",
						"execution": {
							"platform": "slurm",
							"resources": {
								"partition": "gpu",
								"gres": "gpu:1",
								"modules": [
									"GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
								]
							}
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {}
							},
							"additional": {}
						},
						"input": {
							"generic": [
								{
									"source": "10_npt_grompp",
									"extension": "tpr"
								}
							]
						}
					},
					{
						"step_id": "12_prod_md_grompp",
						"type": "grompp",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {
									"-n": "index.ndx"
								}
							},
							"additional": {
								"fields": {
									"nsteps": "40000"
								},
								"make_ndx_command": "auto",
								"-r":False
							}
						},
						"input": {
							"generic": [
								
								{
									"source":prod_mdp,
									"extension": "mdp"
								}
							]
						}
					},
					{
						"step_id": "13_prod_mdrun",
						"type": "mdrun",
						"execution": {
							"platform": "slurm",
							"resources": {
								"partition": "gpu",
								"gres": "gpu:1",
								"modules": [
									"GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
								]
							}
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {
									"-nb": "gpu",
									"-bonded": "gpu",
									"-pme": "gpu"
								}
							}
						},
						"input": {
							"generic": [
								{
									"source": "12_prod_md_grompp",
									"extension": "tpr"
								}
							]
						}
					},
					{
						"step_id": "14_trjconv",
						"type": "trjconv",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"flags": [
									"-center"
								],
								"parameters": {
									"-pbc": "mol",
									"-n": "index.ndx"
								}
							},
							"additional": {
								"pipe_input": "Protein_Other System"
							}
						}
					},
					{
						"step_id": "14b_trjconv",
						"type": "trjconv",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"flags": [],
								"parameters": {
									"-fit": "rot+trans",
									"-n": "index.ndx"
								}
							},
							"additional": {
								"pipe_input": "Protein_Other System"
							}
						}
					},
					{
						"step_id": "15_gmx_MMPBSA",
						"type": "gmx_mmpbsa",
						"execution": {
							"prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2 && module load gmx_MMPBSA/1.3.3-fosscuda-2019a-Python-3.7.2"
						},
						"settings": {
							"arguments": {
								"parameters": {}
							},
							"additional": {
								"coupling_groups": "Protein Other",
								"forcefield": os.path.join(os.environ.get("GMXLIB"),"amber14sb.ff")
							}
						},
						"input": {
							"generic": [
								{
									"source": "14b_trjconv",
									"extension": "xtc"
								},
								{
									"source": "13_prod_mdrun",
									"extension": "tpr"
								}
								
							]
						},
						"writeout": [
							{
								"generic": {
									"key": "dat"
								},
								"destination": {
									"resource": os.path.join(output_dir, "FINAL_RESULTS_MMPBSA.dat")
								}
							}
						]
					}
				],
				"iter_settings": {
					"iter_mode": "n_iters",
					"n_iters": 3,
					"parallelizer_settings": {
						"parallelize":False,
					}
				}
			}
		]
	}
}
with open(os.path.join(config_dir, "mmgbsa_gmx.json"), 'w') as f:
    json.dump(conf, f, indent=4)



The workflow can be executed by running the following command (with paths ammended as necessary), in a terminal. 

In [7]:
# this run will take a few minutes to complete with a GPU.  You can watch icolos_run.log to check it's progress
conf = os.path.join(config_dir, "mmgbsa_gmx.json")

command = f"icolos -conf {conf} -debug"
subprocess.run(command, shell=True)

WARNING - Could not import module espsim, check it is installed in your environment
                                                                                                                                                 
                                        ooooo   .oooooo.     .oooooo.   ooooo          .oooooo.    .oooooo..o                                         
                                        `888'  d8P'  `Y8b   d8P'  `Y8b  `888'         d8P'  `Y8b  d8P'    `Y8                                         
                                         888  888          888      888  888         888      888 Y88bo.                                              
                                         888  888          888      888  888         888      888  `"Y8888o.                                          
                                         888  888          888      888  888         888      888      `"Y88b                                         
               

CompletedProcess(args='icolos -conf ../config/gromacs/mmgbsa_gmx.json -debug', returncode=0)

You can now view the resulting trajectory files using VMD or similar